[View in Colaboratory](https://colab.research.google.com/github/jadevaibhav/ArtistIdentifier/blob/master/style_loss+siamese.ipynb)

In [1]:
import keras
import keras.backend as K
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout, Reshape,Concatenate
from keras.models import Model, load_model,Sequential
model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape = (224,224,3))

layer_dict = dict([(layer.name, layer) for layer in model.layers])

Using TensorFlow backend.


58892288/58889256 [==============================] - 2s 0us/step


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
import keras
import tensorflow as tf

In [20]:
#vgg.summary()
for layer in model.layers:
  if(layer.name in STYLE_LAYERS):
    print(layer.name)

block1_pool
block2_pool
block3_pool
block4_pool
block5_pool


In [0]:
layer_dict

In [0]:
# GRADED FUNCTION: compute_layer_style_cost

def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from a_G (≈1 line)
    (m,n_H, n_W, n_C) = a_G.get_shape().as_list()
    #print(m,n_C,n_H)
    #print(a_G.shape[1])
    
    #a_G = tf.convert_to_tensor(a_G)
    #a_S = tf.convert_to_tensor(a_S)
    
    #print(tf.transpose(a_S,perm=[0,3,2,1]).shape)
    
    # Reshape the images to have them of shape (n_C, n_H*n_W) (≈2 lines)
    a_S = Reshape((n_C,n_H*n_W))(keras.layers.Permute((3,2,1))(a_S))
    a_G = Reshape((n_C,n_H*n_W))(keras.layers.Permute((3,2,1))(a_G))
    
    #print(a_S.shape)
    # Computing gram_matrices for both images S and G (≈2 lines)
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)
    
    # Computing the loss (≈1 line)
    J_style_layer = tf.reduce_sum(tf.square(tf.subtract(GS,GG))/(4*(n_C**2)*((n_H*n_W)**2)))
    
    ### END CODE HERE ###
    
    return J_style_layer

In [0]:
# GRADED FUNCTION: gram_matrix

def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """
    ### START CODE HERE ### (≈1 line)
    GA = keras.layers.Dot(0)([A,A])
    ### END CODE HERE ###
    
    return GA

In [0]:
STYLE_LAYERS = [
    'block1_pool', 
    'block2_pool', 
    'block3_pool', 
    'block4_pool',
    'block5_pool']

In [0]:
def multi_model(model):
  
  output = []
  
  for layer_name in STYLE_LAYERS:
    
    out = model.get_layer(layer_name).output
    #print(type(out))
    #out = Reshape((out.shape[1],out.shape[2],out.shape[3]))(out)
    #print(type(out))
    output.append(out) 
  #output = tf.convert_to_tensor(output)
  m = Model(inputs= model.input ,outputs=output)
  return m

In [0]:
def art_model(in_shape,model):
  
  mod = multi_model(model)
  
  in1 = Input(in_shape)
  in2 = Input(in_shape)
  #print(type(in1))
  a_a = mod(in1)
  a_s = mod(in2)
  
  #print(a_s[0].shape)
  #print(a_s[0])
  
  out1 = Concatenate(axis = 1)([a_a[0],a_s[0]])
  out2 = Concatenate(axis = 1)([a_a[1],a_s[1]])
  out3 = Concatenate(axis = 1)([a_a[2],a_s[2]])
  out4 = Concatenate(axis = 1)([a_a[3],a_s[3]])
  out5 = Concatenate(axis = 1)([a_a[4],a_s[4]])
  
  #print(out1.shape)
  m = Model(inputs = [in1,in2],outputs = [out1,out2,out3,out4,out5])
  return m

In [0]:
import tensorflow as tf
def style_loss(y_true, y_pred):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    #anchor, positive, negative = y_pred[:,0,:], y_pred[:,1,:], y_pred[:,2,:]
    
    #print(y_pred.shape)
    shape = y_pred.shape
    l = int(int(shape[1])*(0.5))
    y1 = y_pred[:,0:l,:,:]
    y2 = y_pred[:,l:,:,:]
    #print(y1)    

    J_style = compute_layer_style_cost(y1,y2)
    
    #for i in range(len(STYLE_LAYERS)):
      
      #cost = compute_layer_style_cost(y_pred[i,0,:,:],y_pred[i,1,:,:])
      #J_style += cost
   
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    #pos_dist = tf.reduce_sum(tf.square(anchor - positive))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    #neg_dist = tf.reduce_sum(tf.square(anchor - negative))
    # Step 3: subtract the two previous distances and add alpha.
    #basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    #loss = tf.maximum(basic_loss, 0)
    ### END CODE HERE ###
    
    return J_style

In [10]:
m = art_model((224,224,3),model)
m.compile(optimizer= 'adam', loss = style_loss , metrics = None)
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 112, 112, 64 14714688    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 224, 112, 64) 0           model_1[1][0]                    
          

In [0]:
import cv2
import random

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir('drive/nnfl/NNFL')

In [15]:
!ls

Aleksey_Savrasov      Michelangelo	   siam_resnet.h5
artist_siamese.ipynb  Pablo_Picasso	   siam_resnet_with_softmax.h5
Claude_Monet	      Paul_Cezanne	   siam+style_vgg_art_model.h5
code		      Pierre_Renoir	   style_loss+siamese.ipynb
comparison.ipynb      preprocessing.ipynb  test.h5
data.h5		      Rembrandt		   tranfer_resnet.h5
data_x.h5	      resnet50+dropout.h5  transfer_resnet50.ipynb
data_x.pickle	      resnet50.h5	   Van_Gogh
Da_Vinci	      resnet50.ipynb	   Wassily_Kandisky
Frida_Kahlo	      Salvador_Dali


In [16]:
import os
path = os.listdir('.')
path.sort(key=str)
fold = path[:12]
#print(fold)

for j in range(100):
  X1 = []
  X2 = []
  #X3 = []
  for i in range(50):
      a = np.random.randint(0,len(fold))
      fold1 = os.listdir(fold[a])
      fold1.sort(key=str.lower)
      #fold2 = os.listdir(fold[b])
      #fold2.sort(key=str.lower)
      #print(len(fold1))
      c,d = random.sample(range(0,len(fold1[:-1])),2)
      #e = np.random.randint(0,len(fold2[:-1]))
      #print(c==d)
      img1 = np.asarray(cv2.imread(fold[a]+'/'+fold1[c],1))
      ran = np.random.randint(0,100)
      img1 = img1[ran:ran+224,ran:ran+224] 
      img2 = np.asarray(cv2.imread(fold[a]+'/'+fold1[d],1))
      ran = np.random.randint(0,100)
      img2 = img2[ran:ran+224,ran:ran+224]
      #img3 = np.asarray(cv2.imread(fold[b]+'/'+fold2[e],1))
      #ran = np.random.randint(0,100)
      #img3 = img3[ran:ran+224,ran:ran+224]
      #print(img1.shape,img2.shape,img3.shape)
      #print(img1.all() == img2.all())
      if(len(img1.shape) == 3 and len(img2.shape) == 3):
        X1.append(img1)
        X2.append(img2)  
        #X3.append(img3)
        
  X1 = np.asarray(X1)
  #print(X1.shape)
  X2 = np.asarray(X2)
  #X3 = np.asarray(X3)
  
  y1 = np.zeros((50, 112, 112, 64))
  y2 = np.zeros((50, 56, 56, 128))
  y3 = np.zeros((50, 28, 28, 256))
  y4 = np.zeros((50, 14, 14, 512))
  y5 = np.zeros((50, 7, 7, 512))
  
  y = [y1,y2,y3,y4,y5]
  #print(y.shape)
  m.fit([X1,X2],y,epochs=5,batch_size=4)
  m.save('siam+style_vgg_art_model.h5')

Epoch 1/5
50/50 [==============================] - 25s 501ms/step - loss: 19011487744.9600 - concatenate_1_loss: 3822980093.4400 - concatenate_2_loss: 7673351469.0450 - concatenate_3_loss: 7511736194.5580 - concatenate_4_loss: 3419255.5637 - concatenate_5_loss: 206.5560
Epoch 2/5
50/50 [==============================] - 18s 370ms/step - loss: 1851470781.4400 - concatenate_1_loss: 1841390533.1200 - concatenate_2_loss: 10080275.7856 - concatenate_3_loss: 15.9021 - concatenate_4_loss: 3.5070e-07 - concatenate_5_loss: 1.2548e-07
Epoch 3/5
50/50 [==============================] - 18s 370ms/step - loss: 1377977564.1600 - concatenate_1_loss: 1376996685.4400 - concatenate_2_loss: 980876.9369 - concatenate_3_loss: 1.1067 - concatenate_4_loss: 2.8871e-07 - concatenate_5_loss: 5.2823e-08
Epoch 4/5
50/50 [==============================] - 18s 370ms/step - loss: 1075929428.4800 - concatenate_1_loss: 1075535198.7200 - concatenate_2_loss: 394237.7369 - concatenate_3_loss: 0.4680 - concatenate_4_loss:

ValueError: ignored